In [1]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth

# JIRA API credentials
# base_url = "https://corecard.atlassian.net/"
api_token = ""
email = ""

# API endpoint to fetch issues
endpoint = f"{base_url}rest/api/3/search"
headers = {
    "Accept": "application/json"
}

# JQL query
jql = "project = CCP AND issuetype = Story AND fixVersion = R40 AND status not in (Cancelled, Backlog)"
params = {
    "jql": jql,
    "maxResults": 100, 
    "startAt": 0        # Starting index for pagination
}

# List to store all issues
all_issues = []

try:
    while True:
        # Request
        response = requests.get(
            endpoint,
            headers=headers,
            params=params,
            auth=HTTPBasicAuth(email, api_token)
        )

        if response.status_code == 200:
            data = response.json()

            # Extracting issues from the response
            issues = data.get('issues', [])
            all_issues.extend(issues)

            # Checking if there are more issues to fetch
            total = data.get('total', 0)
            start_at = params['startAt']
            max_results = params['maxResults']
            
            if start_at + max_results >= total:
                break  # All issues have been fetched

            # Update startAt to fetch the next page
            params['startAt'] += max_results
        else:
            print(f"Failed to fetch data from JIRA API. Status Code: {response.status_code}")
            print(f"Response: {response.text}")
            break

except Exception as e:
    print(f"An error occurred: {e}")

# Prepare data for DataFrame
df_data = []
for issue in all_issues:
    key = issue.get('key', '')
    fields = issue.get('fields', {})

    # Extracting the required fields
    summary = fields.get('summary', '')
    status = fields.get('status', {})
    status_name = status.get('name', '') if status else ''
    assignee = fields.get('assignee', {})
    assignee_name = assignee.get('displayName', 'Unassigned') if assignee else 'Unassigned'
    created = fields.get('created', '-')
    fix_versions = fields.get('fixVersions', [])
    fix_versions_names = [version.get('name', '') for version in fix_versions]
    fix_versions_str = ', '.join(fix_versions_names)
    parent = fields.get('parent', {})
    parent_key = parent.get('key', '-')
    # Custom fields
    ba = fields.get('customfield_10112', [])
    ba_manager = ba[0].get('displayName') if ba else None
    ba_res = fields.get('customfield_10113', [])
    ba_resource = ba_res[0].get('displayName') if ba_res else None
    dev = fields.get('customfield_10114', [])
    dev_manager = dev[0].get('displayName') if dev else None
    dev_resource = fields.get('customfield_10115', [])
    development_resource = dev_resource[0].get('displayName') if dev_resource else None
    t_manager = fields.get('customfield_10116', [])
    test_manager = t_manager[0].get('displayName') if t_manager else None
    t_resource = fields.get('customfield_10117', [])
    test_resource = t_resource[0].get('displayName') if t_resource else None
    u_manager = fields.get('customfield_10118', [])
    uat_manager = u_manager[0].get('displayName') if u_manager else None
    u_resource = fields.get('customfield_10119', [])
    uat_resource = u_resource[0].get('displayName') if u_resource else None
    baeffort = fields.get('customfield_10122', None)
    ba_effort = baeffort
    qaeffort = fields.get('customfield_10125', None)
    qa_effort = qaeffort

    # Appending the extracted information as a dictionary to the list
    df_data.append({
        'Key': key,
        'Summary': summary,
        'Status': status_name,
        'Assignee': assignee_name,
        'Created': created,
        'Fix Version': fix_versions_str,
        'Parent Key': parent_key,
        'BA Manager': ba_manager,
        'BA Resource': ba_resource,
        'Dev Manager': dev_manager,
        'Dev Resource': development_resource,
        'Test Manager': test_manager,
        'Test Resource': test_resource,
        'UAT Manager': uat_manager,
        'UAT Resource': uat_resource,
        'BA Effort': ba_effort,
        'QA Effort': qa_effort
    })

# Creating a Master DataFrame from the extracted data
df = pd.DataFrame(df_data)


In [3]:
response.status_code

200